# Notebook to open TXLA model output with xroms. 
Write a short function to open a single file or a list of files 

In [3]:
#Packages
import xgcm
from xgcm import Grid
import xarray as xr
import xroms
import glob

/d2/home/dylan/.conda/envs/txla_model/lib/python3.11/site-packages/pyproj/__init__.py:89: UserWarning: pyproj unable to set database path.
  _pyproj_global_context_initialize()


# Open model output.
## Single file, then list of files. We start with a diagnostic file because xroms isn't compatible. For this, use simple xarray.
### Notes: 
    - To open model output on hafen, go to https://hafen.geos.tamu.edu/thredds/catalog/catalog.html. 
    - When entering the model output path, click the desired subdirectory, then 'OPeNDAP, then copy+paste the url and remove '.html' from the url
    - This works for nearly all paths on Hafen, with the exception of model hindcast.
        -It doesn't raise an error code, but won't run the cell. I think this is because the hindcast is so large that even opening with xarray is dangerous. Don't have evidence though. 

In [4]:
path = 'http://hafen.geos.tamu.edu:8080/thredds/dodsC/NcML/txla_nest_p1hr_dia_agg'
ds = xr.open_dataset(path)
ds

<xarray.Dataset>
Dimensions:         (tracer: 5, s_rho: 30, s_w: 31, eta_rho: 191, xi_rho: 671,
                     eta_u: 191, xi_u: 670, eta_v: 190, xi_v: 671,
                     eta_psi: 190, xi_psi: 670, ocean_time: 1008)
Coordinates:
  * s_rho           (s_rho) float64 -0.9833 -0.95 -0.9167 ... -0.05 -0.01667
  * s_w             (s_w) float64 -1.0 -0.9667 -0.9333 ... -0.06667 -0.03333 0.0
    lon_rho         (eta_rho, xi_rho) float64 ...
    lat_rho         (eta_rho, xi_rho) float64 ...
    lon_u           (eta_u, xi_u) float64 ...
    lat_u           (eta_u, xi_u) float64 ...
    lon_v           (eta_v, xi_v) float64 ...
    lat_v           (eta_v, xi_v) float64 ...
    lon_psi         (eta_psi, xi_psi) float64 ...
    lat_psi         (eta_psi, xi_psi) float64 ...
  * ocean_time      (ocean_time) datetime64[ns] 2010-06-02T18:30:00 ... 2010-...
Dimensions without coordinates: tracer, eta_rho, xi_rho, eta_u, xi_u, eta_v,
                                xi_v, eta_psi, xi_psi
Data variables: (12/150)
    ntimes          int32 ...
    ndtfast         int32 ...
    dt              float64 ...
    dtfast          float64 ...
    dstart          datetime64[ns] ...
    shuffle         int32 ...
    ...              ...
    salt_hdiff      (ocean_time, s_rho, eta_rho, xi_rho) float32 ...
    salt_xdiff      (ocean_time, s_rho, eta_rho, xi_rho) float32 ...
    salt_ydiff      (ocean_time, s_rho, eta_rho, xi_rho) float32 ...
    salt_sdiff      (ocean_time, s_rho, eta_rho, xi_rho) float32 ...
    salt_vdiff      (ocean_time, s_rho, eta_rho, xi_rho) float32 ...
    salt_rate       (ocean_time, s_rho, eta_rho, xi_rho) float32 ...
Attributes: (12/31)
    file:                      ./outputs/2010/NM/ver0/ocean_dia_00005.nc
    format:                    netCDF-4/HDF5 file
    Conventions:               CF-1.4, SGRID-0.3
    type:                      ROMS/TOMS diagnostics file
    title:                     TXLA ROMS with numerical mixing
    var_info:                  varinfo_01032022.dat
    ...                        ...
    compiler_flags:            -frepack-arrays -Wno-argument-mismatch -O3 -ff...
    tiling:                    006x002
    history:                   ROMS/TOMS, Version 3.9, Friday - January 28, 2...
    ana_file:                  ROMS/Functionals/ana_btflux.h, ROMS/Functional...
    CPP_options:               TXLA2_NM, ANA_BPFLUX, ANA_BSFLUX, ANA_BTFLUX, ...
    EXTRA_DIMENSION.boundary:  4

# Open parent history and average file from the 2021 ver 2.0 nested runs.

In [5]:
paths = ['https://hafen.geos.tamu.edu/thredds/dodsC/NcML/txla_nest_p_his_2021_v2_agg', # Parent, his file
         'https://hafen.geos.tamu.edu/thredds/dodsC/NcML/txla_nest_p_avg_2021_v2_agg'] # Parent, avg file
 
def open_roms(path):
    ds1 = xroms.open_netcdf(path)
    ds1, grid1 = xroms.roms_dataset(ds1)
    return ds1, grid1

ds = []
grid = []
for i in range(len(paths)):
    ds1, grid1 = open_roms(paths[i])
    ds.append(ds1)
    grid.append(grid1)

In [6]:
ds[0]

<xarray.Dataset>
Dimensions:          (tracer: 5, s_rho: 30, s_w: 31, eta_rho: 191, xi_rho: 671,
                      xi_u: 670, eta_v: 190, ocean_time: 721)
Coordinates: (12/23)
  * s_rho            (s_rho) float64 -0.9833 -0.95 -0.9167 ... -0.05 -0.01667
  * s_w              (s_w) float64 -1.0 -0.9667 -0.9333 ... -0.03333 0.0
    lon_rho          (eta_rho, xi_rho) float64 dask.array<chunksize=(191, 671), meta=np.ndarray>
    lat_rho          (eta_rho, xi_rho) float64 dask.array<chunksize=(191, 671), meta=np.ndarray>
    lon_u            (eta_rho, xi_u) float64 dask.array<chunksize=(191, 670), meta=np.ndarray>
    lat_u            (eta_rho, xi_u) float64 dask.array<chunksize=(191, 670), meta=np.ndarray>
    ...               ...
    z_w_v            (ocean_time, s_w, eta_v, xi_rho) float64 dask.array<chunksize=(1, 31, 190, 671), meta=np.ndarray>
    z_w_psi          (ocean_time, s_w, eta_v, xi_u) float64 dask.array<chunksize=(1, 31, 190, 670), meta=np.ndarray>
    z_rho            (ocean_time, s_rho, eta_rho, xi_rho) float64 dask.array<chunksize=(1, 30, 191, 671), meta=np.ndarray>
    z_rho_u          (ocean_time, s_rho, eta_rho, xi_u) float64 dask.array<chunksize=(1, 30, 191, 670), meta=np.ndarray>
    z_rho_v          (ocean_time, s_rho, eta_v, xi_rho) float64 dask.array<chunksize=(1, 30, 190, 671), meta=np.ndarray>
    z_rho_psi        (ocean_time, s_rho, eta_v, xi_u) float64 dask.array<chunksize=(1, 30, 190, 670), meta=np.ndarray>
Dimensions without coordinates: tracer
Data variables: (12/138)
    ntimes           int32 ...
    ndtfast          int32 ...
    dt               float64 ...
    dtfast           float64 ...
    dstart           datetime64[ns] ...
    shuffle          int32 ...
    ...               ...
    dz_w_u           (ocean_time, s_w, eta_rho, xi_u) float64 dask.array<chunksize=(1, 31, 191, 670), meta=np.ndarray>
    dz_v             (ocean_time, s_rho, eta_v, xi_rho) float64 dask.array<chunksize=(1, 30, 190, 671), meta=np.ndarray>
    dz_w_v           (ocean_time, s_w, eta_v, xi_rho) float64 dask.array<chunksize=(1, 31, 190, 671), meta=np.ndarray>
    dz_psi           (ocean_time, s_rho, eta_v, xi_u) float64 dask.array<chunksize=(1, 30, 190, 670), meta=np.ndarray>
    dz_w_psi         (ocean_time, s_w, eta_v, xi_u) float64 dask.array<chunksize=(1, 31, 190, 670), meta=np.ndarray>
    dA               (eta_rho, xi_rho) float64 dask.array<chunksize=(191, 671), meta=np.ndarray>
Attributes: (12/32)
    file:                      ./outputs/2021/nest/ocean_his_parent_rst1_0000...
    format:                    netCDF-4/HDF5 file
    Conventions:               CF-1.4, SGRID-0.3
    type:                      ROMS/TOMS history file
    title:                     TXLA ROMS 2-way nesting with numerical mixing
    var_info:                  varinfo_01032022.dat
    ...                        ...
    tiling:                    012x010
    history:                   ROMS/TOMS, Version 3.9, Friday - February 3, 2...
    ana_file:                  ROMS/Functionals/ana_btflux.h, ROMS/Functional...
    CPP_options:               TXLA2_NEST, ANA_BPFLUX, ANA_BSFLUX, ANA_BTFLUX...
    EXTRA_DIMENSION.N:         30
    EXTRA_DIMENSION.boundary:  4

In [7]:
grid[0]

<xgcm.Grid>
X Axis (not periodic, boundary=None):
  * center   xi_rho --> inner
  * inner    xi_u --> center
Y Axis (not periodic, boundary=None):
  * center   eta_rho --> inner
  * inner    eta_v --> center
Z Axis (not periodic, boundary=None):
  * center   s_rho --> outer
  * outer    s_w --> center

In [8]:
ds[1]

<xarray.Dataset>
Dimensions:          (tracer: 5, s_rho: 30, s_w: 31, eta_rho: 191, xi_rho: 671,
                      xi_u: 670, eta_v: 190, ocean_time: 720)
Coordinates: (12/23)
  * s_rho            (s_rho) float64 -0.9833 -0.95 -0.9167 ... -0.05 -0.01667
  * s_w              (s_w) float64 -1.0 -0.9667 -0.9333 ... -0.03333 0.0
    lon_rho          (eta_rho, xi_rho) float64 dask.array<chunksize=(191, 671), meta=np.ndarray>
    lat_rho          (eta_rho, xi_rho) float64 dask.array<chunksize=(191, 671), meta=np.ndarray>
    lon_u            (eta_rho, xi_u) float64 dask.array<chunksize=(191, 670), meta=np.ndarray>
    lat_u            (eta_rho, xi_u) float64 dask.array<chunksize=(191, 670), meta=np.ndarray>
    ...               ...
    z_w_v            (ocean_time, s_w, eta_v, xi_rho) float64 dask.array<chunksize=(1, 31, 190, 671), meta=np.ndarray>
    z_w_psi          (ocean_time, s_w, eta_v, xi_u) float64 dask.array<chunksize=(1, 31, 190, 670), meta=np.ndarray>
    z_rho            (ocean_time, s_rho, eta_rho, xi_rho) float64 dask.array<chunksize=(1, 30, 191, 671), meta=np.ndarray>
    z_rho_u          (ocean_time, s_rho, eta_rho, xi_u) float64 dask.array<chunksize=(1, 30, 191, 670), meta=np.ndarray>
    z_rho_v          (ocean_time, s_rho, eta_v, xi_rho) float64 dask.array<chunksize=(1, 30, 190, 671), meta=np.ndarray>
    z_rho_psi        (ocean_time, s_rho, eta_v, xi_u) float64 dask.array<chunksize=(1, 30, 190, 670), meta=np.ndarray>
Dimensions without coordinates: tracer
Data variables: (12/147)
    ntimes           int32 ...
    ndtfast          int32 ...
    dt               float64 ...
    dtfast           float64 ...
    dstart           datetime64[ns] ...
    shuffle          int32 ...
    ...               ...
    dz_w_u           (ocean_time, s_w, eta_rho, xi_u) float64 dask.array<chunksize=(1, 31, 191, 670), meta=np.ndarray>
    dz_v             (ocean_time, s_rho, eta_v, xi_rho) float64 dask.array<chunksize=(1, 30, 190, 671), meta=np.ndarray>
    dz_w_v           (ocean_time, s_w, eta_v, xi_rho) float64 dask.array<chunksize=(1, 31, 190, 671), meta=np.ndarray>
    dz_psi           (ocean_time, s_rho, eta_v, xi_u) float64 dask.array<chunksize=(1, 30, 190, 670), meta=np.ndarray>
    dz_w_psi         (ocean_time, s_w, eta_v, xi_u) float64 dask.array<chunksize=(1, 31, 190, 670), meta=np.ndarray>
    dA               (eta_rho, xi_rho) float64 dask.array<chunksize=(191, 671), meta=np.ndarray>
Attributes: (12/31)
    file:                      ./outputs/2021/nest/ocean_avg_parent_rst1_0000...
    format:                    netCDF-4/HDF5 file
    Conventions:               CF-1.4, SGRID-0.3
    type:                      ROMS/TOMS nonlinear model averages file
    title:                     TXLA ROMS 2-way nesting with numerical mixing
    var_info:                  varinfo_01032022.dat
    ...                        ...
    compiler_flags:            -fp-model fast -heap-arrays -ip -O3 -traceback
    tiling:                    012x010
    history:                   ROMS/TOMS, Version 3.9, Friday - February 3, 2...
    ana_file:                  ROMS/Functionals/ana_btflux.h, ROMS/Functional...
    CPP_options:               TXLA2_NEST, ANA_BPFLUX, ANA_BSFLUX, ANA_BTFLUX...
    EXTRA_DIMENSION.boundary:  4

In [9]:
grid[1]

<xgcm.Grid>
X Axis (not periodic, boundary=None):
  * center   xi_rho --> inner
  * inner    xi_u --> center
Y Axis (not periodic, boundary=None):
  * center   eta_rho --> inner
  * inner    eta_v --> center
Z Axis (not periodic, boundary=None):
  * center   s_rho --> outer
  * outer    s_w --> center

# Try 2022 outputs to be sure they work 

In [10]:
paths = ['https://hafen.geos.tamu.edu/thredds/catalog/catalog.html?dataset=txla_nest_c_his_2022_agg', # Parent, his file
         'https://hafen.geos.tamu.edu/thredds/catalog/catalog.html?dataset=txla_nest_c_avg_2022_agg'] # Parent, avg file
 
def open_roms(path):
    ds1 = xroms.open_netcdf(path)
    ds1, grid1 = xroms.roms_dataset(ds1)
    return ds1, grid1

ds = []
grid = []
for i in range(len(paths)):
    ds1, grid1 = open_roms(paths[i])
    ds.append(ds1)
    grid.append(grid1)

OSError: [Errno -75] NetCDF: Malformed or unexpected Constraint: 'https://hafen.geos.tamu.edu/thredds/catalog/catalog.html?dataset=txla_nest_c_his_2022_agg'

# 2022 output doesn't appear to work?